# Train Dataset Augmentation

## Import

In [ ]:
import os

import random

import librosa
import soundfile

import torch
import torchaudio

from IPython.display import Audio

import pandas as pd
from tqdm.notebook import tqdm

## Load Data

In [ ]:
df = pd.read_csv("./data/train.csv")

In [ ]:
df['id']

In [ ]:
df['label']

## Augmentation Function

In [ ]:
def generate(id1, id2, from_pth="./data/train", to_pth="./data/train_augmented", filename=None, sample_rate=32000, channel_size=1):
    # Load Audio Files
    if "NONE_" not in id1:
        waveform1, sample_rate1 = torchaudio.load(from_pth + "/" + id1 + ".ogg")
    else:
        waveform1, sample_rate1 = torch.zeors((channel_size, 44000)), sample_rate
    if "NONE_" not in id2:
        waveform2, sample_rate2 = torchaudio.load(from_pth + "/" + id2 + ".ogg")
    else:
        waveform2, sample_rate2 = torch.zeros((channel_size, 44000)), sample_rate
    
    if sample_rate1 != sample_rate or sample_rate2 != sample_rate:
        raise ValueError("Sampling rate does not match.")
    
    waveform_size = max(waveform1.size(1), waveform2.size(1))
    
    # Extend waveform size
    waveform1 = torch.nn.functional.pad(waveform1, (0, waveform_size - waveform1.size(1)))
    waveform2 = torch.nn.functional.pad(waveform2, (0, waveform_size - waveform2.size(1)))
    
    # Create Random White Noise
    noise = torch.zeros((channel_size, 0))
    while noise.size(1) < waveform_size:
        partition = torch.randn((channel_size, random.randint(1, waveform_size//5)))
        if random.choice([True, False]):
            partition *= random.uniform(0, 0.1)  # select noise volume
        else:  # remove noise
            partition *= 0
        noise = torch.cat((noise, partition), dim=1)
    noise = noise[:, :waveform_size]
    
    # Combine Waveforms
    waveform_noise = waveform1 + waveform2 + noise
    
    # Save Augmented Audio
    if filename is None:
        filename = id1 + "-" + id2
    filepath = to_pth + "/" + filename + ".ogg"
    torchaudio.save(filepath, waveform_noise, sample_rate)
    return filepath

### Augmentation 1: Add Noise to Waveforms

In [ ]:
for count in range(3):
    target_dir = f"./data/train_noise_type{count+1}"
    if not os.path.isdir(target_dir):
        os.mkdir(target_dir)

    df = pd.read_csv("./data/train.csv")

    for i in tqdm(range(len(df))):
        df['path'][i] = generate(df['id'][i], "NONE_", from_pth="./data/train", to_pth=target_dir, filename=df['id'][i]).replace("./data", ".")
    
    df.to_csv(target_dir + ".csv", index=False)

In [ ]:
if not os.path.isdir("./data/train_augmented"):
    os.mkdir("./data/train_augmented")